In [1]:
# Celda 0
from google.colab import drive
drive.mount('/content/drive')

import os, sys, platform, torch, random, numpy as np
print(f"Python: {sys.version.split()[0]} | PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()} | device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

# Semillas
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Python: 3.12.11 | PyTorch: 2.8.0+cu126
CUDA disponible: True | device: cuda


In [2]:
# Celda 1
BASE_DIR      = "/content/drive/MyDrive/CognitivaAI"
DATA_DIR      = os.path.join(BASE_DIR, "oas1_data")                   # contiene DATA/OAS1_PROCESSED/*.png + CSV originales
ARTIFACTS_DIR = os.path.join(BASE_DIR, "oas1_resnet18_linearprobe")   # carpeta de trabajo/resultados
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

print("BASE_DIR     :", BASE_DIR)
print("DATA_DIR     :", DATA_DIR)
print("ARTIFACTS_DIR:", ARTIFACTS_DIR)

def data_path(*x):       return os.path.join(DATA_DIR, *x)
def artifacts_path(*x):  return os.path.join(ARTIFACTS_DIR, *x)


BASE_DIR     : /content/drive/MyDrive/CognitivaAI
DATA_DIR     : /content/drive/MyDrive/CognitivaAI/oas1_data
ARTIFACTS_DIR: /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe


In [3]:
# Celda 2
expected = [
    data_path("oas1_train.csv"),
    data_path("oas1_val.csv"),
    data_path("oas1_test.csv"),
]
for p in expected:
    print(("OK" if os.path.exists(p) else "FALTA"), "→", p)

# Directorio con PNGs (antiguamente los CSV referencian "DATA/OAS1_PROCESSED/...")
png_root_guess = data_path("DATA", "OAS1_PROCESSED")
print("Posible carpeta de PNGs:", png_root_guess, "| existe:", os.path.exists(png_root_guess))


OK → /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_train.csv
OK → /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_val.csv
OK → /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_test.csv
Posible carpeta de PNGs: /content/drive/MyDrive/CognitivaAI/oas1_data/DATA/OAS1_PROCESSED | existe: False


In [4]:
from pathlib import Path

DATA_DIR = Path("/content/drive/MyDrive/CognitivaAI/oas1_data")

pngs_lower = list(DATA_DIR.rglob("*.png"))
pngs_upper = list(DATA_DIR.rglob("*.PNG"))
all_pngs  = pngs_lower + pngs_upper

print(f"PNG encontrados (recursivo): {len(all_pngs)}")
for p in all_pngs[:5]:
    print("  •", p)


PNG encontrados (recursivo): 8320
  • /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0404_MR1_slice18.png
  • /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0404_MR1_slice13.png
  • /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0404_MR1_slice01.png
  • /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0404_MR1_slice19.png
  • /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0404_MR1_slice00.png


In [5]:
import pandas as pd
from pathlib import Path
from collections import defaultdict

BASE_DIR      = Path("/content/drive/MyDrive/CognitivaAI")
DATA_DIR      = BASE_DIR/"oas1_data"
CSV_TRAIN_SRC = DATA_DIR/"oas1_train_colab.csv"
CSV_VAL_SRC   = DATA_DIR/"oas1_val_colab.csv"
CSV_TEST_SRC  = DATA_DIR/"oas1_test_colab.csv"

# 1) Indexar TODO lo que sea PNG (independiente del subdirectorio y del case)
all_pngs = list(DATA_DIR.rglob("*.png")) + list(DATA_DIR.rglob("*.PNG"))
print(f"[Index] PNG totales encontrados: {len(all_pngs)}")
if len(all_pngs) == 0:
    raise SystemExit("⛔ No se encontraron PNG en DATA_DIR. Revisa el montaje de Drive o la ubicación de 'oas1_data'.")

# 2) Mapa basename -> lista de paths completos (por si hay duplicados)
index_by_base = defaultdict(list)
for p in all_pngs:
    index_by_base[p.name].append(p)

# 3) Función de reparación: reemplaza la columna 'png_path' por el path real encontrado
def repair_csv(csv_in: Path, csv_out: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_in)
    if 'png_path' not in df.columns:
        raise ValueError(f"{csv_in} no tiene columna 'png_path'")

    new_paths = []
    missing   = []

    for p_old in df['png_path'].astype(str).tolist():
        base = Path(p_old).name  # nos quedamos con el basename
        candidates = index_by_base.get(base, [])
        if candidates:
            # si hay varios con el mismo basename, elegimos el 1º (habitual: sólo 1)
            new_paths.append(str(candidates[0]))
        else:
            new_paths.append(None)
            missing.append(base)

    df['png_path'] = new_paths
    n_missing = sum(p is None for p in new_paths)
    if n_missing > 0:
        print(f"⚠ {csv_in.name}: faltantes tras reparar = {n_missing}")
        print("   Ejemplos:", missing[:5])
    else:
        print(f"✅ {csv_in.name}: todas las rutas reparadas")

    df.to_csv(csv_out, index=False)
    print(f"   → Guardado: {csv_out}")
    return df

CSV_TRAIN_FIXED = DATA_DIR/"oas1_train_colab_mapped.csv"
CSV_VAL_FIXED   = DATA_DIR/"oas1_val_colab_mapped.csv"
CSV_TEST_FIXED  = DATA_DIR/"oas1_test_colab_mapped.csv"

df_tr = repair_csv(CSV_TRAIN_SRC, CSV_TRAIN_FIXED)
df_va = repair_csv(CSV_VAL_SRC,   CSV_VAL_FIXED)
df_te = repair_csv(CSV_TEST_SRC,  CSV_TEST_FIXED)

print("\nResumen shapes:",
      f"train={df_tr.shape}, val={df_va.shape}, test={df_te.shape}")


[Index] PNG totales encontrados: 8320
✅ oas1_train_colab.csv: todas las rutas reparadas
   → Guardado: /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_train_colab_mapped.csv
✅ oas1_val_colab.csv: todas las rutas reparadas
   → Guardado: /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_val_colab_mapped.csv
✅ oas1_test_colab.csv: todas las rutas reparadas
   → Guardado: /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_test_colab_mapped.csv

Resumen shapes: train=(2820, 6), val=(940, 6), test=(940, 6)


In [6]:
from pathlib import Path

def count_missing(csv_path: Path) -> int:
    df = pd.read_csv(csv_path)
    miss = 0
    for p in df['png_path']:
        if not Path(p).exists():
            miss += 1
    return miss

for csv in [CSV_TRAIN_FIXED, CSV_VAL_FIXED, CSV_TEST_FIXED]:
    miss = count_missing(csv)
    print(f"{csv.name} → faltantes: {miss}")


oas1_train_colab_mapped.csv → faltantes: 0
oas1_val_colab_mapped.csv → faltantes: 0
oas1_test_colab_mapped.csv → faltantes: 0


In [7]:
# === 4) DATASETS & DATALOADERS (Colab) ===
import pandas as pd, numpy as np, torch, os
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

BASE_DIR      = Path("/content/drive/MyDrive/CognitivaAI")
DATA_DIR      = BASE_DIR/"oas1_data"
ARTIFACTS_DIR = BASE_DIR/"oas1_resnet18_linearprobe"
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

CSV_TRAIN = str(DATA_DIR/"oas1_train_colab_mapped.csv")
CSV_VAL   = str(DATA_DIR/"oas1_val_colab_mapped.csv")
CSV_TEST  = str(DATA_DIR/"oas1_test_colab_mapped.csv")
print("Usando CSV mapeados:\n", CSV_TRAIN, "\n", CSV_VAL, "\n", CSV_TEST)

IMG_SIZE = 224

train_tf = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
eval_tf = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

class PNGCsvDataset(Dataset):
    def __init__(self, csv_path, transform):
        self.df = pd.read_csv(csv_path)
        # Asegurar tipos
        self.df["png_path"] = self.df["png_path"].astype(str)
        self.df["target"]   = self.df["target"].astype(int)
        self.transform = transform

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        p   = Path(row["png_path"])
        if not p.exists():
            raise FileNotFoundError(f"No puedo leer {p}")
        img = Image.open(p).convert("RGB")
        x   = self.transform(img)
        y   = int(row["target"])
        return x, torch.tensor(y, dtype=torch.long)

ds_tr = PNGCsvDataset(CSV_TRAIN, train_tf)
ds_va = PNGCsvDataset(CSV_VAL,   eval_tf)
ds_te = PNGCsvDataset(CSV_TEST,  eval_tf)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Workers/pin_memory para T4
num_workers = 4 if device.type == "cuda" else 2
dl_tr = DataLoader(ds_tr, batch_size=32, shuffle=True,  num_workers=num_workers, pin_memory=(device.type=="cuda"), persistent_workers=True)
dl_va = DataLoader(ds_va, batch_size=64, shuffle=False, num_workers=num_workers, pin_memory=(device.type=="cuda"), persistent_workers=True)
dl_te = DataLoader(ds_te, batch_size=64, shuffle=False, num_workers=num_workers, pin_memory=(device.type=="cuda"), persistent_workers=True)

print(f"[OK] Train={len(ds_tr)}  Val={len(ds_va)}  Test={len(ds_te)}")


Usando CSV mapeados:
 /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_train_colab_mapped.csv 
 /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_val_colab_mapped.csv 
 /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_test_colab_mapped.csv
Device: cuda
[OK] Train=2820  Val=940  Test=940


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
# === 5) SANITY CHECK ===
import torch.nn as nn
from torchvision import models

# Modelo ligero (ResNet18)
m = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
m.fc = nn.Linear(m.fc.in_features, 2)
m = m.to(device)

# Un batch
xb, yb = next(iter(dl_tr))
xb, yb = xb.to(device), yb.to(device)
with torch.no_grad():
    logits = m(xb)
print("Sanity logits shape:", logits.shape)  # (B,2) esperado

# Mini-overfit a 1 batch (5-10 pasos) para comprobar que baja la loss
opt = torch.optim.AdamW(m.parameters(), lr=3e-4, weight_decay=1e-4)
crit = nn.CrossEntropyLoss()
m.train()
for step in range(10):
    opt.zero_grad(set_to_none=True)
    out = m(xb)
    loss = crit(out, yb)
    loss.backward()
    opt.step()
    print(f"step={step:02d} loss={loss.item():.4f}")


Sanity logits shape: torch.Size([32, 2])
step=00 loss=0.7878
step=01 loss=0.1106
step=02 loss=0.0202
step=03 loss=0.0078
step=04 loss=0.0038
step=05 loss=0.0022
step=06 loss=0.0014
step=07 loss=0.0009
step=08 loss=0.0006
step=09 loss=0.0004


In [9]:
# Celda A — Diagnóstico (muestra 10 rutas del loader actual y comprueba existencia)
import os, itertools

def peek_loader_paths(loader, n=10):
    ds = getattr(loader, "dataset", None)
    if ds is None or not hasattr(ds, "df"):
        print("❌ Este loader no tiene .dataset.df; probablemente no está inicializado correctamente.")
        return
    print("CSV usado por este dataset (primeras 3 filas):")
    print(ds.df.head(3))
    paths = ds.df["png_path"].tolist()[:n]
    print(f"\nPrimeras {n} rutas:")
    for p in paths:
        print(" -", p, "| exists:", os.path.exists(p))

print("— TRAIN —")
try: peek_loader_paths(train_loader, n=10)
except NameError: print("train_loader no existe en memoria.")

print("\n— VAL —")
try: peek_loader_paths(val_loader, n=10)
except NameError: print("val_loader no existe en memoria.")

print("\n— TEST —")
try: peek_loader_paths(test_loader, n=10)
except NameError: print("test_loader no existe en memoria.")


— TRAIN —
train_loader no existe en memoria.

— VAL —
val_loader no existe en memoria.

— TEST —
test_loader no existe en memoria.


In [10]:
# Celda B — Loaders robustos desde cero (Colab)
import os, glob, json
from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np

# 1) Paths base (ajusta si cambiaste carpetas)
BASE_DIR      = Path("/content/drive/MyDrive/CognitivaAI")
DATA_DIR      = BASE_DIR / "oas1_data"
ARTIFACTS_DIR = BASE_DIR / "oas1_resnet18_linearprobe"

CSV_TRAIN = DATA_DIR / "oas1_train_colab_mapped.csv"
CSV_VAL   = DATA_DIR / "oas1_val_colab_mapped.csv"
CSV_TEST  = DATA_DIR / "oas1_test_colab_mapped.csv"

assert CSV_TRAIN.exists() and CSV_VAL.exists() and CSV_TEST.exists(), "Faltan CSV *_colab_mapped.csv"

# 2) Índice global (filename -> ruta completa real)
pngs = glob.glob(str(DATA_DIR / "*.png"))
index_by_name = {Path(p).name: p for p in pngs}
print(f"[Index] PNG totales encontrados: {len(index_by_name)}")

# 3) Dataset robusto (normaliza y aplica fallbacks)
from torchvision import transforms

def default_transform():
    return transforms.Compose([
        transforms.Resize((224,224), interpolation=Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std=[0.229,0.224,0.225]),
    ])

class DatasetRobusto(Dataset):
    def __init__(self, csv_path, transform=None, index_by_name=None):
        self.df = pd.read_csv(csv_path)
        self.df["png_path"] = self.df["png_path"].astype(str)
        self.df["target"] = self.df["target"].astype(int)
        self.transform = transform or default_transform()
        self.index_by_name = index_by_name or {}
        self.data_dir = Path(DATA_DIR)

    def _resolve_path(self, p):
        # 1) normalizar separadores
        p = str(p).replace("\\", "/")
        # 2) si es relativo con prefijos viejos, quedarnos con solo el nombre
        name = Path(p).name
        # 3) si existe tal cual, bien
        if os.path.exists(p):
            return p
        # 4) si existe bajo DATA_DIR + nombre, usarlo
        cand = str(self.data_dir / name)
        if os.path.exists(cand):
            return cand
        # 5) buscar en índice por nombre
        if name in self.index_by_name and os.path.exists(self.index_by_name[name]):
            return self.index_by_name[name]
        # 6) último intento: si la ruta está dentro de DATA_DIR con subcarpetas (no es tu caso)
        #    podrías implementar una búsqueda recursiva, pero sería más lenta
        raise FileNotFoundError(f"No puedo resolver la ruta del PNG: {p}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        p = self._resolve_path(row["png_path"])
        y = int(row["target"])

        with Image.open(p) as im:
            im = im.convert("RGB")
        x = self.transform(im)
        return x, torch.tensor(y, dtype=torch.long)

# 4) Instancias de datasets y loaders
train_ds = DatasetRobusto(CSV_TRAIN, index_by_name=index_by_name)
val_ds   = DatasetRobusto(CSV_VAL,   index_by_name=index_by_name)
test_ds  = DatasetRobusto(CSV_TEST,  index_by_name=index_by_name)

BATCH_SIZE   = 32
NUM_WORKERS  = 2 if torch.cuda.is_available() else 0
PIN_MEMORY   = torch.cuda.is_available()

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, drop_last=False)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, drop_last=False)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, drop_last=False)

# 5) Smoke test: tomar 1 batch y verificar
xb, yb = next(iter(train_loader))
print(f"[Smoke test] batch shapes: x={tuple(xb.shape)} y={tuple(yb.shape)}  (OK si no hubo excepciones)")


[Index] PNG totales encontrados: 8320
[Smoke test] batch shapes: x=(32, 3, 224, 224) y=(32,)  (OK si no hubo excepciones)


In [11]:
# Celda C — Lista 10 rutas resueltas y existencia real
for i in range(10):
    p = train_ds._resolve_path(train_ds.df.iloc[i]["png_path"])
    print(i, "→", p, "| exists:", os.path.exists(p))


0 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice00.png | exists: True
1 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice01.png | exists: True
2 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice02.png | exists: True
3 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice03.png | exists: True
4 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice04.png | exists: True
5 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice05.png | exists: True
6 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice06.png | exists: True
7 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice07.png | exists: True
8 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice08.png | exists: True
9 → /content/drive/MyDrive/CognitivaAI/oas1_data/OAS1_0001_MR1_slice09.png | exists: True


In [12]:
# ============================
# Celda 6 — Entrenamiento (Colab, T4)
#  - Warm-up (congelar backbone) + fine-tuning
#  - Early stopping por AUC de validación
#  - Guardado "solo pesos" + metadatos JSON (evita UnpicklingError)
#  - AMP (mixed precision) en GPU
# Requisitos previos:
#   • train_loader, val_loader, test_loader ya creados (celdas previas)
#   • ARTIFACTS_DIR definido
# ============================

import os, json, math, time
from pathlib import Path
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torchvision import models
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    # En Ampere (T4) mejora algo el rendimiento numérico de matmul
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass

# ---------- Paths de checkpoints (solo pesos + metadatos) ----------
CKPT_DIR = Path(ARTIFACTS_DIR) / "ckpts_colab"
CKPT_DIR.mkdir(parents=True, exist_ok=True)
best_weights = CKPT_DIR / "resnet18_best_val_auc_weights.pth"
best_meta    = CKPT_DIR / "resnet18_best_val_auc_meta.json"

# ---------- Modelo (ResNet18 ligera para CPU/GPU) ----------
# Nota: usamos pesos ImageNet para acelerar convergencia
def create_model(num_classes=2, pretrained=True):
    weights = models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None
    model = models.resnet18(weights=weights)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

model = create_model().to(device)

# ---------- Pérdida ----------
# Usamos CrossEntropy con un poco de label smoothing (mejor calibración/robustez)
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)

# ---------- Optimizador y Scheduler ----------
LR = 1e-4
WEIGHT_DECAY = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# En PyTorch recientes, ReduceLROnPlateau acepta 'verbose', pero para compatibilidad lo omitimos
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=2
)

# ---------- Utilidad de evaluación ----------
@torch.no_grad()
def evaluate(loader, model, device):
    model.eval()
    all_probs, all_labels, all_preds = [], [], []
    running_loss, running_n = 0.0, 0

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        logits = model(x)
        loss = criterion(logits, y)

        prob = torch.softmax(logits, dim=1)[:, 1]
        pred = torch.argmax(logits, dim=1)

        running_loss += loss.item() * y.size(0)
        running_n    += y.size(0)

        all_probs.append(prob.detach().cpu().numpy())
        all_labels.append(y.detach().cpu().numpy())
        all_preds.append(pred.detach().cpu().numpy())

    if running_n == 0:
        return {"loss": np.nan, "acc": np.nan, "auc": np.nan, "prauc": np.nan}

    all_probs  = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)
    all_preds  = np.concatenate(all_preds)

    acc = accuracy_score(all_labels, all_preds)
    try:
        auc = roc_auc_score(all_labels, all_probs) if len(np.unique(all_labels)) > 1 else np.nan
    except Exception:
        auc = np.nan
    try:
        prauc = average_precision_score(all_labels, all_probs) if len(np.unique(all_labels)) > 1 else np.nan
    except Exception:
        prauc = np.nan

    loss = running_loss / running_n
    return {"loss": loss, "acc": acc, "auc": auc, "prauc": prauc}

# ---------- Config de entrenamiento ----------
EPOCHS_TOTAL   = 12          # con T4 suele ir rápido; ajusta si lo ves estable
EPOCHS_WARMUP  = 2           # congelamos backbone al principio
PATIENCE_ES    = 4           # early stopping si no mejora el AUC de validación
best_auc       = -np.inf
patience_count = 0

# ---------- Warm-up: congelar backbone (solo entrenar la FC) ----------
for name, p in model.named_parameters():
    if not name.startswith("fc."):
        p.requires_grad = False

print("Device:", device.type)
print("Warm-up congelado.")

# AMP: mixed precision para GPU; en CPU no aporta
scaler = torch.amp.GradScaler(device.type, enabled=(device.type == "cuda"))

# ---------- Loop de entrenamiento ----------
for epoch in range(1, EPOCHS_TOTAL + 1):
    t0 = time.time()
    model.train()
    running_loss, running_n = 0.0, 0

    # Pasar a fine-tuning (descongelar) después del warm-up
    if epoch == (EPOCHS_WARMUP + 1):
        for p in model.parameters():
            p.requires_grad = True
        # Re-inicializamos optimizer (mismo LR) tras descongelar
        optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        print("→ Fine-tuning activado (backbone descongelado)")

    # --------- Entrenamiento por batches ---------
    for x, y in tqdm(train_loader, desc=f"[Epoch {epoch}/{EPOCHS_TOTAL}] Train", leave=False):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        # AMP autocast solo en GPU
        with torch.autocast(device_type=device.type, dtype=torch.float16, enabled=(device.type == "cuda")):
            logits = model(x)
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * y.size(0)
        running_n    += y.size(0)

    train_loss = running_loss / max(1, running_n)

    # --------- Evaluación (train y val) ---------
    train_metrics = evaluate(train_loader, model, device)
    val_metrics   = evaluate(val_loader,   model, device)

    # Scheduler según AUC de validación
    if not math.isnan(val_metrics["auc"]):
        scheduler.step(val_metrics["auc"])

    t1 = time.time()
    print(f"""
Epoch {epoch:02d}/{EPOCHS_TOTAL} | {t1 - t0:.1f}s
  Train: loss={train_loss:.4f} | acc={train_metrics['acc']:.3f} | AUC={train_metrics['auc']:.3f} | PR-AUC={train_metrics['prauc']:.3f}
  Val  : loss={val_metrics['loss']:.4f} | acc={val_metrics['acc']:.3f} | AUC={val_metrics['auc']:.3f} | PR-AUC={val_metrics['prauc']:.3f}
""".strip())

    # --------- Checkpoint si mejora AUC ---------
    cur_auc = val_metrics["auc"]
    if not math.isnan(cur_auc) and cur_auc > best_auc:
        best_auc = cur_auc
        patience_count = 0

        # Guardamos SOLO los pesos y un JSON con metadatos
        torch.save(model.state_dict(), best_weights)
        with open(best_meta, "w") as f:
            json.dump({"epoch": int(epoch), "best_auc": float(best_auc)}, f, indent=2)

        print(f"\n  ✅ Nuevo mejor AUC val = {best_auc:.4f} → {best_weights}")
    else:
        patience_count += 1
        print(f"\n  ↪ Sin mejora AUC (patience {patience_count}/{PATIENCE_ES})")

    # --------- Early stopping ---------
    if patience_count >= PATIENCE_ES:
        print("\n🛑 Early stopping.")
        break

# ---------- Cargar mejor checkpoint y evaluar en TEST ----------
if best_weights.exists():
    # Carga segura: "solo pesos" evita el UnpicklingError de PyTorch 2.6+
    model.load_state_dict(torch.load(best_weights, map_location=device))
    try:
        meta = json.loads(best_meta.read_text())
        print(f"\nCargado mejor checkpoint (AUC val={meta.get('best_auc', float('nan')):.4f})")
    except Exception:
        print("\nCargado mejor checkpoint (metadatos no disponibles)")
else:
    print("\n⚠ No se encontró checkpoint. Se evalúa el último estado del modelo.")

test_metrics = evaluate(test_loader, model, device)
print(f"""
=== TEST (por PNG) ===
  Loss   : {test_metrics['loss']:.4f}
  Acc    : {test_metrics['acc']:.3f}
  ROC-AUC: {test_metrics['auc']:.3f}
  PR-AUC : {test_metrics['prauc']:.3f}
""".strip())


Device: cuda
Warm-up congelado.


Epoch 01/12 | 74.8s
  Train: loss=0.7096 | acc=0.533 | AUC=0.542 | PR-AUC=0.465
  Val  : loss=0.7132 | acc=0.516 | AUC=0.525 | PR-AUC=0.448

  ✅ Nuevo mejor AUC val = 0.5246 → /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/ckpts_colab/resnet18_best_val_auc_weights.pth


Epoch 02/12 | 38.7s
  Train: loss=0.6923 | acc=0.575 | AUC=0.569 | PR-AUC=0.490
  Val  : loss=0.6965 | acc=0.564 | AUC=0.531 | PR-AUC=0.449

  ✅ Nuevo mejor AUC val = 0.5311 → /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/ckpts_colab/resnet18_best_val_auc_weights.pth
→ Fine-tuning activado (backbone descongelado)


Epoch 03/12 | 39.5s
  Train: loss=0.5578 | acc=0.880 | AUC=0.965 | PR-AUC=0.956
  Val  : loss=0.7455 | acc=0.616 | AUC=0.656 | PR-AUC=0.568

  ✅ Nuevo mejor AUC val = 0.6562 → /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/ckpts_colab/resnet18_best_val_auc_weights.pth


Epoch 04/12 | 40.8s
  Train: loss=0.2848 | acc=0.971 | AUC=0.997 | PR-AUC=0.996
  Val  : loss=0.8571 | acc=0.635 | AUC=0.636 | PR-AUC=0.574

  ↪ Sin mejora AUC (patience 1/4)


Epoch 05/12 | 40.4s
  Train: loss=0.1870 | acc=0.973 | AUC=1.000 | PR-AUC=1.000
  Val  : loss=1.2197 | acc=0.609 | AUC=0.633 | PR-AUC=0.553

  ↪ Sin mejora AUC (patience 2/4)


Epoch 06/12 | 38.8s
  Train: loss=0.1619 | acc=0.995 | AUC=1.000 | PR-AUC=1.000
  Val  : loss=1.0093 | acc=0.640 | AUC=0.642 | PR-AUC=0.584

  ↪ Sin mejora AUC (patience 3/4)


Epoch 07/12 | 39.3s
  Train: loss=0.1403 | acc=1.000 | AUC=1.000 | PR-AUC=1.000
  Val  : loss=0.9037 | acc=0.619 | AUC=0.630 | PR-AUC=0.561

  ↪ Sin mejora AUC (patience 4/4)

🛑 Early stopping.

Cargado mejor checkpoint (AUC val=0.6562)
=== TEST (por PNG) ===
  Loss   : 0.8172
  Acc    : 0.621
  ROC-AUC: 0.642
  PR-AUC : 0.535


In [13]:
# Celda 7 — Eval a nivel paciente con el mejor checkpoint
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

# reutiliza 'model', 'device', 'val_loader', 'test_loader', 'val_ds', 'test_ds'
# y el path 'best_weights' que ya definiste en tu celda de entrenamiento

# 1) Cargar los mejores pesos (por si el kernel se reinició)
model.load_state_dict(torch.load(best_weights, map_location=device))
model.eval()

@torch.no_grad()
def predict_png(loader, ds, device):
    """Devuelve un DF alineado con ds.df (assume shuffle=False)."""
    probs, preds, labels = [], [], []
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        logits = model(xb)
        prob1 = torch.softmax(logits, dim=1)[:, 1]
        pred  = torch.argmax(logits, dim=1)
        probs.append(prob1.cpu().numpy())
        preds.append(pred.cpu().numpy())
        labels.append(yb.cpu().numpy())
    prob = np.concatenate(probs)
    pred = np.concatenate(preds)
    lab  = np.concatenate(labels)

    df = ds.df.copy().reset_index(drop=True)
    # si no existe 'scan_id' en el CSV, lo extraemos del filename
    if "scan_id" not in df.columns:
        df["scan_id"] = df["png_path"].apply(lambda p: Path(p).name.split("_slice")[0])
    df["prob1"] = prob
    df["pred"]  = pred
    df["target"] = lab
    return df

def metrics_bin(y_true, score, hard=None):
    acc = accuracy_score(y_true, (hard if hard is not None else (score>=0.5).astype(int)))
    try:
        auc = roc_auc_score(y_true, score) if len(np.unique(y_true))>1 else np.nan
    except Exception:
        auc = np.nan
    try:
        pr  = average_precision_score(y_true, score) if len(np.unique(y_true))>1 else np.nan
    except Exception:
        pr = np.nan
    return acc, auc, pr

def aggregate_patient(df_png):
    """Agrega por scan_id: prob_media. Toma 1er target (debería ser consistente)."""
    g = df_png.groupby("scan_id", as_index=False).agg(
        prob_mean=("prob1","mean"),
        target=("target","first")
    )
    g["pred"] = (g["prob_mean"]>=0.5).astype(int)
    return g

# 2) PNG-level
val_png = predict_png(val_loader, val_ds, device)
tst_png = predict_png(test_loader, test_ds, device)

val_acc, val_auc, val_pr = metrics_bin(val_png["target"].values, val_png["prob1"].values, val_png["pred"].values)
tst_acc, tst_auc, tst_pr = metrics_bin(tst_png["target"].values, tst_png["prob1"].values, tst_png["pred"].values)

print(f"[VAL-PNG ] Acc={val_acc:.3f} | AUC={val_auc:.3f} | PR-AUC={val_pr:.3f}")
print(f"[TEST-PNG] Acc={tst_acc:.3f} | AUC={tst_auc:.3f} | PR-AUC={tst_pr:.3f}")

# 3) Paciente-level
val_pat = aggregate_patient(val_png)
tst_pat = aggregate_patient(tst_png)

vA, vU, vP = metrics_bin(val_pat["target"].values, val_pat["prob_mean"].values, val_pat["pred"].values)
tA, tU, tP = metrics_bin(tst_pat["target"].values, tst_pat["prob_mean"].values, tst_pat["pred"].values)

print(f"[VAL-PACIENTE ] n={len(val_pat)} | Acc={vA:.3f} | AUC={vU:.3f} | PR-AUC={vP:.3f}")
print(f"[TEST-PACIENTE] n={len(tst_pat)} | Acc={tA:.3f} | AUC={tU:.3f} | PR-AUC={tP:.3f}")

# 4) Guardar resultados
out_dir = Path(ARTIFACTS_DIR) / "patient_eval_colab"
out_dir.mkdir(parents=True, exist_ok=True)
val_png.to_csv(out_dir/"val_png_preds.csv", index=False)
tst_png.to_csv(out_dir/"test_png_preds.csv", index=False)
val_pat.to_csv(out_dir/"val_patient_preds.csv", index=False)
tst_pat.to_csv(out_dir/"test_patient_preds.csv", index=False)
print("CSV guardados en:", out_dir)


[VAL-PNG ] Acc=0.616 | AUC=0.656 | PR-AUC=0.568
[TEST-PNG] Acc=0.621 | AUC=0.642 | PR-AUC=0.535
[VAL-PACIENTE ] n=47 | Acc=0.681 | AUC=0.704 | PR-AUC=0.627
[TEST-PACIENTE] n=47 | Acc=0.723 | AUC=0.676 | PR-AUC=0.578
CSV guardados en: /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/patient_eval_colab


In [14]:
# Celda 8 — Extracción de embeddings (512-D) y guardado a .npz
from torchvision import models

# 1) reconstruir el mismo modelo y cargar pesos
full = models.resnet18(weights=None)
full.fc = nn.Linear(full.fc.in_features, 2)  # igual que en training
full.load_state_dict(torch.load(best_weights, map_location=device))
full.to(device).eval()

# 2) backbone = todo menos la FC; salida [B,512,1,1] → flatten [B,512]
backbone = nn.Sequential(*list(full.children())[:-1]).to(device).eval()
for p in backbone.parameters(): p.requires_grad = False

@torch.no_grad()
def extract_embeddings(loader, ds):
    feats, labs = [], []
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        f = backbone(xb)            # [B,512,1,1]
        f = torch.flatten(f, 1)     # [B,512]
        feats.append(f.cpu().numpy())
        labs.append(yb.numpy())
    X = np.concatenate(feats, axis=0).astype(np.float32)
    y = np.concatenate(labs,  axis=0).astype(np.int64)
    return X, y

# IMPORTANT: loaders con shuffle=False para alinear con .df si luego quieres mapear scan_id
train_loader_eval = DataLoader(train_ds, batch_size=64, shuffle=False,
                               num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
val_loader_eval   = DataLoader(val_ds,   batch_size=64, shuffle=False,
                               num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_loader_eval  = DataLoader(test_ds,  batch_size=64, shuffle=False,
                               num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

X_tr, y_tr = extract_embeddings(train_loader_eval, train_ds)
X_va, y_va = extract_embeddings(val_loader_eval,   val_ds)
X_te, y_te = extract_embeddings(test_loader_eval,  test_ds)

emb_dir = Path(ARTIFACTS_DIR) / "embeddings_npz_colab"
emb_dir.mkdir(parents=True, exist_ok=True)
npz_path = emb_dir / "oas1_resnet18_backbone_embeddings.npz"
np.savez_compressed(npz_path, X_train=X_tr, y_train=y_tr, X_val=X_va, y_val=y_va, X_test=X_te, y_test=y_te)

print("Embeddings:",
      f"train={X_tr.shape}, val={X_va.shape}, test={X_te.shape}")
print("Guardado en:", npz_path)


Embeddings: train=(2820, 512), val=(940, 512), test=(940, 512)
Guardado en: /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/embeddings_npz_colab/oas1_resnet18_backbone_embeddings.npz


In [15]:
# Celda 9 — Linear probe (Logistic Regression) + métricas por PNG y por paciente
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 1) cargar embeddings (por si ejecutas en otra sesión)
npz = np.load(npz_path)
X_tr, y_tr = npz["X_train"], npz["y_train"]
X_va, y_va = npz["X_val"],   npz["y_val"]
X_te, y_te = npz["X_test"],  npz["y_test"]

# 2) LR con escalado
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=2000, solver="lbfgs", n_jobs=1))
]).fit(X_tr, y_tr)

# 3) Probabilidades a nivel PNG
p_va = clf.predict_proba(X_va)[:,1]
p_te = clf.predict_proba(X_te)[:,1]

pred_va = (p_va>=0.5).astype(int)
pred_te = (p_te>=0.5).astype(int)

def metrics(y, proba, pred):
    acc = accuracy_score(y, pred)
    try: auc = roc_auc_score(y, proba) if len(np.unique(y))>1 else np.nan
    except: auc = np.nan
    try: pr  = average_precision_score(y, proba) if len(np.unique(y))>1 else np.nan
    except: pr = np.nan
    return acc, auc, pr

va_acc, va_auc, va_pr = metrics(y_va, p_va, pred_va)
te_acc, te_auc, te_pr = metrics(y_te, p_te, pred_te)

print(f"[VAL-PNG]  Acc={va_acc:.3f} | AUC={va_auc:.3f} | PR-AUC={va_pr:.3f}")
print(f"[TEST-PNG] Acc={te_acc:.3f} | AUC={te_auc:.3f} | PR-AUC={te_pr:.3f}")

# 4) Paciente-level (usamos el orden de val_ds/test_ds porque loaders_eval tenían shuffle=False)
def patient_from_df(df):
    if "scan_id" not in df.columns:
        df = df.copy()
        df["scan_id"] = df["png_path"].apply(lambda p: Path(p).name.split("_slice")[0])
    return df

val_df = val_ds.df.copy().reset_index(drop=True)
tst_df = test_ds.df.copy().reset_index(drop=True)
val_df = patient_from_df(val_df);   val_df["prob1"] = p_va; val_df["pred"] = pred_va
tst_df = patient_from_df(tst_df);   tst_df["prob1"] = p_te; tst_df["pred"] = pred_te

def aggregate(df):
    g = df.groupby("scan_id", as_index=False).agg(
        prob_mean=("prob1","mean"),
        target   =("target","first")
    )
    g["pred"] = (g["prob_mean"]>=0.5).astype(int)
    return g

val_pat = aggregate(val_df)
tst_pat = aggregate(tst_df)

vA, vU, vP = metrics(val_pat["target"].values, val_pat["prob_mean"].values, val_pat["pred"].values)
tA, tU, tP = metrics(tst_pat["target"].values, tst_pat["prob_mean"].values, tst_pat["pred"].values)

print(f"[VAL-PACIENTE]  n={len(val_pat)} | Acc={vA:.3f} | AUC={vU:.3f} | PR-AUC={vP:.3f}")
print(f"[TEST-PACIENTE] n={len(tst_pat)} | Acc={tA:.3f} | AUC={tU:.3f} | PR-AUC={tP:.3f}")


[VAL-PNG]  Acc=0.621 | AUC=0.624 | PR-AUC=0.531
[TEST-PNG] Acc=0.627 | AUC=0.661 | PR-AUC=0.528
[VAL-PACIENTE]  n=47 | Acc=0.553 | AUC=0.719 | PR-AUC=0.623
[TEST-PACIENTE] n=47 | Acc=0.681 | AUC=0.715 | PR-AUC=0.605


# Conclusiones (hasta embeddings + LR):
- El fine-tuning de ResNet18 en Colab (OASIS-1) alcanza un AUC paciente ~0.68.
- La agregación a nivel paciente es fundamental: los resultados por PNG son más bajos.
- El enfoque de embeddings + Linear Probe mejora la estabilidad y generaliza mejor:
  • VAL-PACIENTE AUC ≈ 0.72
  • TEST-PACIENTE AUC ≈ 0.72
- Esto confirma que ResNet18 captura representaciones útiles, y que un clasificador lineal posterior
  puede dar métricas similares o mejores con mucho menor coste de entrenamiento.


In [17]:
# ============================================
# Calibración (Colab) — LR vs LR + Isotónica
#  - Carga embeddings (.npz)
#  - Entrena LR base y Calibrated (isotónica)
#  - Curvas de calibración y Brier score
#  - Agregado a nivel paciente (umbral recall≥0.90 en VAL)
#  - Guarda modelos, plots y CSVs
# ============================================

import os, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import (
    accuracy_score, roc_auc_score, average_precision_score,
    brier_score_loss, precision_recall_curve, precision_score, recall_score
)
import joblib

# ---------- Rutas base (Colab/Drive) ----------
BASE_DIR      = Path("/content/drive/MyDrive/CognitivaAI")
DATA_DIR      = BASE_DIR / "oas1_data"
ARTIFACTS_DIR = BASE_DIR / "oas1_resnet18_linearprobe"

EMB_DIR   = ARTIFACTS_DIR / "embeddings_npz_colab"
EMB_PATH  = EMB_DIR / "oas1_resnet18_backbone_embeddings.npz"

CAL_DIR   = ARTIFACTS_DIR / "calibration_colab"
CAL_DIR.mkdir(parents=True, exist_ok=True)

# CSV de mapeo (orden por PNG para paciente-level)
TRAIN_CSV = DATA_DIR / "oas1_train_colab_mapped.csv"
VAL_CSV   = DATA_DIR / "oas1_val_colab_mapped.csv"
TEST_CSV  = DATA_DIR / "oas1_test_colab_mapped.csv"

assert EMB_PATH.exists(), f"No existe el NPZ de embeddings: {EMB_PATH}"
assert TRAIN_CSV.exists() and VAL_CSV.exists() and TEST_CSV.exists(), "Faltan CSV *_colab_mapped.csv"

# ---------- Carga robusta del NPZ ----------
npz = np.load(EMB_PATH, allow_pickle=False)
keys = sorted(list(npz.keys()))
print("NPZ cargado:", EMB_PATH)
print("Claves:", keys)

def get_array(npz, *names):
    for n in names:
        if n in npz:
            return npz[n]
    return None

X_tr = get_array(npz, "X_train", "X_tr", "arr_0")
y_tr = get_array(npz, "y_train", "y_tr", "arr_1")
X_va = get_array(npz, "X_val",   "X_va", "arr_2")
y_va = get_array(npz, "y_val",   "y_va", "arr_3")
X_te = get_array(npz, "X_test",  "X_te", "arr_4")
y_te = get_array(npz, "y_test",  "y_te", "arr_5")

for name, arr in [("X_tr", X_tr), ("y_tr", y_tr), ("X_va", X_va), ("y_va", y_va), ("X_te", X_te), ("y_te", y_te)]:
    if arr is None:
        raise KeyError(f"Falta {name} en el NPZ")

print(f"Shapes: X_tr={X_tr.shape}, X_va={X_va.shape}, X_te={X_te.shape}")

# ---------- LR (sin calibrar) ----------
lr_base = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("lr", LogisticRegression(max_iter=2000, solver="lbfgs", n_jobs=1))
])
lr_base.fit(X_tr, y_tr)

proba_va_base = lr_base.predict_proba(X_va)[:, 1]
proba_te_base = lr_base.predict_proba(X_te)[:, 1]

def safe_auc(y, p):
    try:
        return roc_auc_score(y, p) if len(np.unique(y)) > 1 else np.nan
    except Exception:
        return np.nan

def safe_ap(y, p):
    try:
        return average_precision_score(y, p) if len(np.unique(y)) > 1 else np.nan
    except Exception:
        return np.nan

def report_split(y_true, proba, name="SPLIT"):
    pred = (proba >= 0.5).astype(int)
    acc   = accuracy_score(y_true, pred)
    auc   = safe_auc(y_true, proba)
    prauc = safe_ap(y_true, proba)
    brier = brier_score_loss(y_true, proba)
    print(f"[{name}] Acc={acc:.3f} | AUC={auc:.3f} | PR-AUC={prauc:.3f} | Brier={brier:.4f}")
    return dict(acc=acc, auc=auc, prauc=prauc, brier=brier)

print("\n=== SIN CALIBRAR (LR) ===")
m_val_base = report_split(y_va, proba_va_base, "VAL")
m_tst_base = report_split(y_te, proba_te_base, "TEST")

# ---------- LR + Isotónica ----------
# NOTA: usamos CalibratedClassifierCV con 'prefit=True' (calibra sobre VAL).
# Entrenamos base en TRAIN y calibramos con VAL para emular pipeline "train→val→test" limpio.
lr_iso = CalibratedClassifierCV(base_estimator=lr_base, method="isotonic", cv="prefit")
lr_iso.fit(X_va, y_va)

proba_va_iso = lr_iso.predict_proba(X_va)[:, 1]
proba_te_iso = lr_iso.predict_proba(X_te)[:, 1]

print("\n=== CALIBRADO (LR + Isotónica) ===")
m_val_iso = report_split(y_va, proba_va_iso, "VAL")
m_tst_iso = report_split(y_te, proba_te_iso, "TEST")

# ---------- Curvas de calibración ----------
def plot_calibration(ax, y_true, proba, label):
    frac_pos, mean_pred = calibration_curve(y_true, proba, n_bins=10, strategy="uniform")
    ax.plot(mean_pred, frac_pos, marker="o", linestyle="-", label=label)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
# VAL
axes[0].plot([0, 1], [0, 1], "k--", lw=1)
plot_calibration(axes[0], y_va, proba_va_base, f"LR (Brier={m_val_base['brier']:.3f})")
plot_calibration(axes[0], y_va, proba_va_iso,  f"LR+Iso (Brier={m_val_iso['brier']:.3f})")
axes[0].set_title("Calibración (VAL)")
axes[0].set_xlabel("Prob. media predicha")
axes[0].set_ylabel("Frecuencia observada")
axes[0].legend()

# TEST
axes[1].plot([0, 1], [0, 1], "k--", lw=1)
plot_calibration(axes[1], y_te, proba_te_base, f"LR (Brier={m_tst_base['brier']:.3f})")
plot_calibration(axes[1], y_te, proba_te_iso,  f"LR+Iso (Brier={m_tst_iso['brier']:.3f})")
axes[1].set_title("Calibración (TEST)")
axes[1].set_xlabel("Prob. media predicha")
axes[1].set_ylabel("Frecuencia observada")
axes[1].legend()

fig.tight_layout()
calib_png = CAL_DIR / "calibration_curves_VAL_TEST.png"
fig.savefig(calib_png, dpi=150, bbox_inches="tight")
plt.close(fig)
print(f"\nFiguras guardadas en: {calib_png}")

# ---------- Guardado de modelos y resumen ----------
joblib.dump(lr_base, CAL_DIR / "lr_uncalibrated.joblib")
joblib.dump(lr_iso,  CAL_DIR / "lr_isotonic_calibrated.joblib")

summary = {
    "npz_path": str(EMB_PATH),
    "val_uncalibrated": m_val_base,
    "test_uncalibrated": m_tst_base,
    "val_isotonic": m_val_iso,
    "test_isotonic": m_tst_iso,
}
with open(CAL_DIR / "calibration_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("Modelos y resumen guardados en:", CAL_DIR)

# ===========================================================
# Agregado a nivel de PACIENTE (usando CSV *_colab_mapped.csv)
#   - proba por PNG → media por paciente
#   - selección de umbral en VAL (recall ≥ 0.90)
#   - evaluación en TEST con ese umbral
# ===========================================================

# Cargamos CSVs y comprobamos longitud vs embeddings
df_tr = pd.read_csv(TRAIN_CSV)
df_va = pd.read_csv(VAL_CSV)
df_te = pd.read_csv(TEST_CSV)

# Intentar detectar columna de paciente
def detect_patient_col(df):
    for c in ["patient_id_canon", "patient_id", "scan_id", "scan", "pid"]:
        if c in df.columns:
            return c
    # si solo hay scan_id, usamos prefijo hasta _MR
    if "png_path" in df.columns:
        return None
    return None

def to_patient_id(df):
    col = detect_patient_col(df)
    if col is not None and col in df.columns and col != "scan_id":
        return df[col].astype(str).tolist()
    # fallback: derivar de scan_id si existe; si no, derivar del png_path
    if "scan_id" in df.columns:
        # ejemplo: OAS1_0001_MR1 → paciente OAS1_0001
        return df["scan_id"].astype(str).str.split("_MR", n=1, expand=True)[0].tolist()
    if "png_path" in df.columns:
        return df["png_path"].astype(str).apply(lambda p: Path(p).name.split("_MR")[0]).tolist()
    raise ValueError("No encuentro columnas para derivar patient_id.")

pid_tr = to_patient_id(df_tr)
pid_va = to_patient_id(df_va)
pid_te = to_patient_id(df_te)

assert len(pid_tr) == len(y_tr), "train CSV y embeddings no alinean"
assert len(pid_va) == len(y_va), "val CSV y embeddings no alinean"
assert len(pid_te) == len(y_te), "test CSV y embeddings no alinean"

# Usamos el clasificador calibrado (isotónica) para las probabilidades finales
proba_va = proba_va_iso
proba_te = proba_te_iso

def aggregate_patient_probs(pid_list, probs, labels):
    df = pd.DataFrame({"patient_id": pid_list, "prob": probs, "label": labels})
    # media por paciente
    g = df.groupby("patient_id", as_index=False).agg(prob=("prob", "mean"), label=("label", "first"))
    return g

g_va = aggregate_patient_probs(pid_va, proba_va, y_va)
g_te = aggregate_patient_probs(pid_te, proba_te, y_te)

def patient_metrics(df, thr=0.5, name="SPLIT-PAC"):
    pred = (df["prob"].values >= thr).astype(int)
    y    = df["label"].values.astype(int)
    acc   = accuracy_score(y, pred)
    auc   = safe_auc(y, df["prob"].values)
    prauc = safe_ap(y, df["prob"].values)
    pre   = precision_score(y, pred, zero_division=0)
    rec   = recall_score(y, pred, zero_division=0)
    print(f"[{name}] n={len(df)} | Acc={acc:.3f} | AUC={auc:.3f} | PR-AUC={prauc:.3f} | P={pre:.3f} | R={rec:.3f}")
    return dict(acc=acc, auc=auc, prauc=prauc, precision=pre, recall=rec, n=len(df))

print("\n=== PACIENTE (prob media por paciente, thr=0.5) ===")
pm_va = patient_metrics(g_va, 0.5, "VAL-PACIENTE")
pm_te = patient_metrics(g_te, 0.5, "TEST-PACIENTE")

# Selección de umbral en VAL buscando recall ≥ 0.90 con mejor precision
prec, rec, thr = precision_recall_curve(g_va["label"].values, g_va["prob"].values)
# precision_recall_curve devuelve thresholds de tamaño len(prec)-1
thr_candidates = np.concatenate([[0.0], thr, [1.0]])

best_thr = 0.5
best_prec = -1.0
for t in thr_candidates:
    pr = (g_va["prob"].values >= t).astype(int)
    r  = recall_score(g_va["label"].values, pr, zero_division=0)
    p  = precision_score(g_va["label"].values, pr, zero_division=0)
    if r >= 0.90 and p > best_prec:
        best_prec = p
        best_thr = t

print(f"\n→ Umbral escogido (VAL, recall≥0.90): thr={best_thr:.4f} | precision={best_prec:.3f}")

pm_va_thr = patient_metrics(g_va, best_thr, "VAL-PACIENTE(thr*)")
pm_te_thr = patient_metrics(g_te, best_thr, "TEST-PACIENTE(thr*)")

# Guardar tablas por paciente y resumen JSON
PAT_DIR = CAL_DIR / "patient_level"
PAT_DIR.mkdir(parents=True, exist_ok=True)

g_va.to_csv(PAT_DIR / "val_patient_probs.csv", index=False)
g_te.to_csv(PAT_DIR / "test_patient_probs.csv", index=False)

patient_summary = {
    "val_patient_default_thr0.5": pm_va,
    "test_patient_default_thr0.5": pm_te,
    "chosen_threshold": float(best_thr),
    "val_patient_thr_star": pm_va_thr,
    "test_patient_thr_star": pm_te_thr
}
with open(PAT_DIR / "patient_summary.json", "w") as f:
    json.dump(patient_summary, f, indent=2)

print("\nCSVs por paciente y resumen guardados en:", PAT_DIR)



NPZ cargado: /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/embeddings_npz_colab/oas1_resnet18_backbone_embeddings.npz
Claves: ['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']
Shapes: X_tr=(2820, 512), X_va=(940, 512), X_te=(940, 512)

=== SIN CALIBRAR (LR) ===
[VAL] Acc=0.621 | AUC=0.624 | PR-AUC=0.531 | Brier=0.3492
[TEST] Acc=0.627 | AUC=0.661 | PR-AUC=0.528 | Brier=0.3347

=== CALIBRADO (LR + Isotónica) ===
[VAL] Acc=0.626 | AUC=0.639 | PR-AUC=0.539 | Brier=0.2295
[TEST] Acc=0.629 | AUC=0.656 | PR-AUC=0.537 | Brier=0.2328


/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(



Figuras guardadas en: /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/calibration_colab/calibration_curves_VAL_TEST.png
Modelos y resumen guardados en: /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/calibration_colab

=== PACIENTE (prob media por paciente, thr=0.5) ===
[VAL-PACIENTE] n=47 | Acc=0.617 | AUC=0.730 | PR-AUC=0.641 | P=0.625 | R=0.250
[TEST-PACIENTE] n=47 | Acc=0.638 | AUC=0.719 | PR-AUC=0.610 | P=0.615 | R=0.400

→ Umbral escogido (VAL, recall≥0.90): thr=0.4037 | precision=0.643
[VAL-PACIENTE(thr*)] n=47 | Acc=0.745 | AUC=0.730 | PR-AUC=0.641 | P=0.643 | R=0.900
[TEST-PACIENTE(thr*)] n=47 | Acc=0.638 | AUC=0.719 | PR-AUC=0.610 | P=0.560 | R=0.700

CSVs por paciente y resumen guardados en: /content/drive/MyDrive/CognitivaAI/oas1_resnet18_linearprobe/calibration_colab/patient_level
